In [25]:
from pathlib import Path
from sqlalchemy import create_engine
import pandas as pd
import sqlite3
import json
from census import Census

In [2]:
# Read the Net_generation_for_all_sectors_monthly CSV file 
data = pd.read_csv('Net_generation_for_all_sectors_.csv', sep=',', skiprows=4)

# Read the renewable_energy_consumption_source CSV file
renewable_energy_consumption = pd.read_csv('renewable_energy_consumption_source.csv')

transposed_data = data.transpose()

# Rename columns with values from the description row
transposed_data.columns = transposed_data.iloc[0]

# Drop the non-numeric rows of data
transposed_data = transposed_data.drop(['description','units', 'source key'])

# Delete 'Net generation for all sectors' and 'United States' columns
del transposed_data['Net generation for all sectors']
del transposed_data['United States']

# Rename 'decription' column to 'date'
transposed_data.index.name = 'date'

# Add an index column
transposed_data.reset_index(inplace = True) 

# Display cleaned_data DataFrame the modified data
transposed_data.head()

description,date,United States : all fuels (utility-scale),United States : coal,United States : petroleum liquids,United States : petroleum coke,United States : natural gas,United States : other gases,United States : nuclear,United States : conventional hydroelectric,United States : other renewables,...,United States : all utility-scale solar,United States : geothermal,United States : biomass,United States : wood and wood-derived fuels,United States : other biomass,United States : hydro-electric pumped storage,United States : other,United States : all solar,United States : small-scale solar photovoltaic,United States : all utility-scale solar
0,Jan 2010,360957,173320,3187,1161,74173,909,72569,22383,NaN,...,10,1312,4629,3126,1503,-565,1014,--,--,10
1,Feb 2010,319735,153044,1251,1122,66198,825,65245,20590,NaN,...,33,1159,4277,2895,1382,-351,909,--,--,33
2,Mar 2010,312168,144406,1272,1198,63431,1010,64635,20886,NaN,...,76,1307,4682,3090,1592,-325,1002,--,--,76
3,Apr 2010,287800,126952,1220,1067,64644,943,57611,19097,NaN,...,112,1240,4490,2932,1558,-335,996,--,--,112
4,May 2010,327936,143272,1851,1143,73665,1017,66658,25079,NaN,...,153,1311,4470,2893,1577,-441,1060,--,--,153


In [31]:
# Read the use_renew_sector.xlsx excel file 
sheets = ['Contents', 'Biomass', 'Geothermal', 'Hydropower', 'Solar Energy', 'Wind Energy']

renewable_data = pd.read_excel('use_renew_sector.xlsx', sheet_name=sheets, index_col=None, header=2)

In [53]:
biomass = energy_data['Biomass']
biomass_renewable_energy = biomass[['State',2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]]
biomass_renewable_energy.head()

,State,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,3299,2298,3660,6494,7530,9181,7802,8231,7668,8687
1,AL,193453,194992,213553,203628,192270,194485,198072,196226,193284,187599
2,AR,105562,103550,106124,105604,95088,93707,93283,94536,92332,74385
3,AZ,29754,27209,27187,33300,35397,37709,37953,40107,40871,35235
4,CA,299705,296354,343657,349289,344065,349253,357861,357070,405324,400214


In [55]:
biomass_renewable_energy.rename(columns={2011:'2011(BM)',
                                        2012:'2012(BM)',
                                        2013:'2013(BM)',
                                        2014:'2014(BM)',
                                        2015:'2015(BM)',
                                        2016:'2016(BM)',
                                        2017:'2017(BM)',
                                        2018:'2018(BM)',
                                        2019:'2019(BM)',
                                        2020:'2020(BM)'}).head()

,State,2011(BM),2012(BM),2013(BM),2014(BM),2015(BM),2016(BM),2017(BM),2018(BM),2019(BM),2020(BM)
0,AK,3299,2298,3660,6494,7530,9181,7802,8231,7668,8687
1,AL,193453,194992,213553,203628,192270,194485,198072,196226,193284,187599
2,AR,105562,103550,106124,105604,95088,93707,93283,94536,92332,74385
3,AZ,29754,27209,27187,33300,35397,37709,37953,40107,40871,35235
4,CA,299705,296354,343657,349289,344065,349253,357861,357070,405324,400214


In [60]:
geothermal = energy_data['Geothermal']
geothermal_renewable_energy = geothermal[['State',2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]]
geothermal_renewable_energy.head()

,State,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,214,186,186,186,186,186,186,186,186,186
1,AL,135,141,141,141,141,141,141,141,141,141
2,AR,750,808,808,808,808,808,808,808,808,808
3,AZ,345,345,345,345,345,345,345,345,345,345
4,CA,124092,121269,119556,117214,112818,107867,108579,108375,99263,101789


In [61]:
geothermal_renewable_energy.rename(columns={2011:'2011(GT)',
                                        2012:'2012(GT)',
                                        2013:'2013(GT)',
                                        2014:'2014(GT)',
                                        2015:'2015(GT)',
                                        2016:'2016(GT)',
                                        2017:'2017(GT)',
                                        2018:'2018(GT)',
                                        2019:'2019(GT)',
                                        2020:'2020(GT)'}).head()

,State,2011(GT),2012(GT),2013(GT),2014(GT),2015(GT),2016(GT),2017(GT),2018(GT),2019(GT),2020(GT)
0,AK,214,186,186,186,186,186,186,186,186,186
1,AL,135,141,141,141,141,141,141,141,141,141
2,AR,750,808,808,808,808,808,808,808,808,808
3,AZ,345,345,345,345,345,345,345,345,345,345
4,CA,124092,121269,119556,117214,112818,107867,108579,108375,99263,101789


In [62]:
hydropower = energy_data['Hydropower']
hydropower_renewable_energy = hydropower[['State',2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]]
hydropower_renewable_energy.head()

,State,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,13066,14988,13696,14632,14613,15311,15135,15141,14446,15465
1,AL,86313,70754,123071,90020,91853,64456,85057,101380,101495,117032
2,AR,28738,20921,25329,25102,33243,32944,27102,27374,36795,39720
3,AZ,89135,63918,56432,58179,60874,66144,62908,63524,55213,56316
4,CA,413488,255384,226643,157197,128612,267078,390080,239556,341319,187416


In [63]:
hydropower_renewable_energy.rename(columns={2011:'2011(HP)',
                                        2012:'2012(HP)',
                                        2013:'2013(HP)',
                                        2014:'2014(HP)',
                                        2015:'2015(HP)',
                                        2016:'2016(HP)',
                                        2017:'2017(HP)',
                                        2018:'2018(HP)',
                                        2019:'2019(HP)',
                                        2020:'2020(HP)'}).head()

,State,2011(HP),2012(HP),2013(HP),2014(HP),2015(HP),2016(HP),2017(HP),2018(HP),2019(HP),2020(HP)
0,AK,13066,14988,13696,14632,14613,15311,15135,15141,14446,15465
1,AL,86313,70754,123071,90020,91853,64456,85057,101380,101495,117032
2,AR,28738,20921,25329,25102,33243,32944,27102,27374,36795,39720
3,AZ,89135,63918,56432,58179,60874,66144,62908,63524,55213,56316
4,CA,413488,255384,226643,157197,128612,267078,390080,239556,341319,187416


In [65]:
solar = energy_data['Solar Energy']
solar_renewable_energy = solar[['State',2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]]
solar_renewable_energy.head()

,State,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,3,3,3,3,3,14,20,28,40,62
1,AL,71,77,98,100,101,431,1811,3423,3625,3437
2,AR,84,98,108,111,124,372,442,2116,2254,3209
3,AZ,7393,17784,31499,43562,48205,52832,66282,70733,73207,80075
4,CA,40964,52660,85168,156446,211736,267428,340183,381368,405353,436394


In [68]:
solar_renewable_energy.rename(columns={2011:'2011(S)',
                                        2012:'2012(S)',
                                        2013:'2013(S)',
                                        2014:'2014(S)',
                                        2015:'2015(S)',
                                        2016:'2016(S)',
                                        2017:'2017(S)',
                                        2018:'2018(S)',
                                        2019:'2019(S)',
                                        2020:'2020(S)'}).head()

,State,2011(S),2012(S),2013(S),2014(S),2015(S),2016(S),2017(S),2018(S),2019(S),2020(S)
0,AK,3,3,3,3,3,14,20,28,40,62
1,AL,71,77,98,100,101,431,1811,3423,3625,3437
2,AR,84,98,108,111,124,372,442,2116,2254,3209
3,AZ,7393,17784,31499,43562,48205,52832,66282,70733,73207,80075
4,CA,40964,52660,85168,156446,211736,267428,340183,381368,405353,436394


In [67]:
wind = energy_data['Wind Energy']
wind_renewable_energy = wind[['State',2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]]
wind_renewable_energy.head()

,State,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,121,356,1386,1445,1487,1562,1305,1410,1269,1127
1,AL,0,0,0,0,0,0,0,0,0,0
2,AR,0,0,0,0,0,0,0,0,0,0
3,AZ,2483,5059,4291,4451,4210,4998,5248,4824,4933,5647
4,CA,75317,92821,122335,123546,113906,124661,118074,127590,122228,119083


In [69]:
wind_renewable_energy.rename(columns={2011:'2011(W)',
                                        2012:'2012(W)',
                                        2013:'2013(W)',
                                        2014:'2014(W)',
                                        2015:'2015(W)',
                                        2016:'2016(W)',
                                        2017:'2017(W)',
                                        2018:'2018(W)',
                                        2019:'2019(W)',
                                        2020:'2020(W)'}).head()

,State,2011(W),2012(W),2013(W),2014(W),2015(W),2016(W),2017(W),2018(W),2019(W),2020(W)
0,AK,121,356,1386,1445,1487,1562,1305,1410,1269,1127
1,AL,0,0,0,0,0,0,0,0,0,0
2,AR,0,0,0,0,0,0,0,0,0,0
3,AZ,2483,5059,4291,4451,4210,4998,5248,4824,4933,5647
4,CA,75317,92821,122335,123546,113906,124661,118074,127590,122228,119083


In [3]:
data.head()

,description,units,source key,Jan 2010,Feb 2010,Mar 2010,Apr 2010,May 2010,Jun 2010,Jul 2010,...,Mar 2020,Apr 2020,May 2020,Jun 2020,Jul 2020,Aug 2020,Sep 2020,Oct 2020,Nov 2020,Dec 2020
0,Net generation for all sectors,thousand megawatthours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,United States,NaN,ELEC.GEN..M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,United States : all fuels (utility-scale),thousand megawatthours,ELEC.GEN.ALL-US-99.M,360957,319735,312168,287800,327936,375759,409725,...,309870.0,279846.0,304837.0,351967.0,409871.0,398536.0,333493.0,313703.0,301403.0,344523.0
3,United States : coal,thousand megawatthours,ELEC.GEN.COW-US-99.M,173320,153044,144406,126952,143272,165491,179600,...,50731.0,40675.0,46527.0,65283.0,89709.0,91145.0,68407.0,59805.0,61182.0,78588.0
4,United States : petroleum liquids,thousand megawatthours,ELEC.GEN.PEL-US-99.M,3187,1251,1272,1220,1851,2656,2970,...,691.0,626.0,691.0,818.0,914.0,887.0,755.0,876.0,800.0,940.0


In [4]:
energy_data_df = transposed_data.copy()

In [5]:
energy_data_df = energy_data_df.fillna(0)
energy_data_df = energy_data_df.replace('--',0)
energy_data_df.head()

description,date,United States : all fuels (utility-scale),United States : coal,United States : petroleum liquids,United States : petroleum coke,United States : natural gas,United States : other gases,United States : nuclear,United States : conventional hydroelectric,United States : other renewables,...,United States : all utility-scale solar,United States : geothermal,United States : biomass,United States : wood and wood-derived fuels,United States : other biomass,United States : hydro-electric pumped storage,United States : other,United States : all solar,United States : small-scale solar photovoltaic,United States : all utility-scale solar
0,Jan 2010,360957,173320,3187,1161,74173,909,72569,22383,0,...,10,1312,4629,3126,1503,-565,1014,0.0,0.0,10
1,Feb 2010,319735,153044,1251,1122,66198,825,65245,20590,0,...,33,1159,4277,2895,1382,-351,909,0.0,0.0,33
2,Mar 2010,312168,144406,1272,1198,63431,1010,64635,20886,0,...,76,1307,4682,3090,1592,-325,1002,0.0,0.0,76
3,Apr 2010,287800,126952,1220,1067,64644,943,57611,19097,0,...,112,1240,4490,2932,1558,-335,996,0.0,0.0,112
4,May 2010,327936,143272,1851,1143,73665,1017,66658,25079,0,...,153,1311,4470,2893,1577,-441,1060,0.0,0.0,153


In [6]:
energy_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 21 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   date                                            132 non-null    object 
 1   United States : all fuels (utility-scale)       132 non-null    object 
 2   United States : coal                            132 non-null    object 
 3   United States : petroleum liquids               132 non-null    object 
 4   United States : petroleum coke                  132 non-null    object 
 5   United States : natural gas                     132 non-null    object 
 6   United States : other gases                     132 non-null    object 
 7   United States : nuclear                         132 non-null    object 
 8   United States : conventional hydroelectric      132 non-null    object 
 9   United States : other renewables           

In [7]:
energy_data_df.columns

Index(['date', 'United States : all fuels (utility-scale)',
       'United States : coal', 'United States : petroleum liquids',
       'United States : petroleum coke', 'United States : natural gas',
       'United States : other gases', 'United States : nuclear',
       'United States : conventional hydroelectric',
       'United States : other renewables', 'United States : wind',
       'United States : all utility-scale solar', 'United States : geothermal',
       'United States : biomass',
       'United States : wood and wood-derived fuels',
       'United States : other biomass',
       'United States : hydro-electric pumped storage',
       'United States : other', 'United States : all solar',
       'United States : small-scale solar photovoltaic',
       'United States : all utility-scale solar'],
      dtype='object', name='description')

In [8]:
# energy_data_df.iloc[1:] = energy_data_df.iloc[1:].astype(int)
# energy_data_df.info()

In [9]:
# energy_data_df[['United States : all fuels (utility-scale)', 'United States : coal',
#        'United States : petroleum liquids', 'United States : petroleum coke',
#        'United States : natural gas', 'United States : other gases',
#        'United States : nuclear', 'United States : conventional hydroelectric',
#        'United States : other renewables', 'United States : wind',
#        'United States : all utility-scale solar', 'United States : geothermal',
#        'United States : biomass']] = energy_data_df[['United States : all fuels (utility-scale)', 'United States : coal',
#        'United States : petroleum liquids', 'United States : petroleum coke',
#        'United States : natural gas', 'United States : other gases',
#        'United States : nuclear', 'United States : conventional hydroelectric',
#        'United States : other renewables', 'United States : wind',
#        'United States : all utility-scale solar', 'United States : geothermal',
#        'United States : biomass']].apply(pd.to_numeric)

In [10]:
# energy_data_df[['United States : all fuels (utility-scale)', 
#                 'United States : coal',
#                'United States : other', 'United States : all solar']] = energy_data_df[['United States : all fuels (utility-scale)', 
#                                                            'United States : coal',
#                                                             'United States : other', 'United States : all solar']].astype(int)


In [11]:
energy_data_df['date'] = pd.to_datetime(energy_data_df['date'])
energy_data_df['United States : all fuels (utility-scale)'] = energy_data_df['United States : all fuels (utility-scale)'].astype(int)
energy_data_df['United States : coal'] = energy_data_df['United States : coal'].astype(int)
energy_data_df['United States : petroleum liquids'] = energy_data_df['United States : petroleum liquids'].astype(int)
energy_data_df['United States : petroleum coke'] = energy_data_df['United States : petroleum coke'].astype(int)
energy_data_df['United States : natural gas'] = energy_data_df['United States : natural gas'].astype(int)
energy_data_df['United States : other gases'] = energy_data_df['United States : other gases'].astype(int)
energy_data_df['United States : nuclear'] = energy_data_df['United States : nuclear'].astype(int)
energy_data_df['United States : conventional hydroelectric'] = energy_data_df['United States : conventional hydroelectric'].astype(int)
energy_data_df['United States : wind'] = energy_data_df['United States : wind'].astype(int)
energy_data_df['United States : all utility-scale solar'] = energy_data_df['United States : all utility-scale solar'].astype(int)
energy_data_df['United States : geothermal'] = energy_data_df['United States : geothermal'].astype(int)
energy_data_df['United States : biomass'] = energy_data_df['United States : biomass'].astype(int)
energy_data_df['United States : wood and wood-derived fuels'] = energy_data_df['United States : wood and wood-derived fuels'].astype(int)
energy_data_df['United States : other biomass'] = energy_data_df['United States : other biomass'].astype(int)
energy_data_df['United States : hydro-electric pumped storage'] = energy_data_df['United States : hydro-electric pumped storage'].astype(int)
energy_data_df['United States : other'] = energy_data_df['United States : other'].astype(int)
energy_data_df['United States : all solar'] = energy_data_df['United States : all solar'].astype(int)
energy_data_df['United States : small-scale solar photovoltaic'] = energy_data_df['United States : small-scale solar photovoltaic'].astype(int)
energy_data_df['United States : all utility-scale solar'] = energy_data_df['United States : all utility-scale solar'].astype(int)

energy_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 21 columns):
 #   Column                                          Non-Null Count  Dtype         
---  ------                                          --------------  -----         
 0   date                                            132 non-null    datetime64[ns]
 1   United States : all fuels (utility-scale)       132 non-null    int64         
 2   United States : coal                            132 non-null    int64         
 3   United States : petroleum liquids               132 non-null    int64         
 4   United States : petroleum coke                  132 non-null    int64         
 5   United States : natural gas                     132 non-null    int64         
 6   United States : other gases                     132 non-null    int64         
 7   United States : nuclear                         132 non-null    int64         
 8   United States : conventional hydroelectric      13

/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_12152/3354050356.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  energy_data_df['date'] = pd.to_datetime(energy_data_df['date'])


In [12]:
energy_data_df.iloc[0][0]

Timestamp('2010-01-01 00:00:00')

In [24]:
year_2010 = energy_data_df[(energy_data_df['date'] >= '2010-01-01') & (energy_data_df['date'] < '2011-01-01')]
year_2011 = energy_data_df[(energy_data_df['date'] >= '2011-01-01') & (energy_data_df['date'] < '2012-01-01')]
year_2012 = energy_data_df[(energy_data_df['date'] >= '2012-01-01') & (energy_data_df['date'] < '2013-01-01')]
year_2013 = energy_data_df[(energy_data_df['date'] >= '2013-01-01') & (energy_data_df['date'] < '2014-01-01')]
year_2014 = energy_data_df[(energy_data_df['date'] >= '2014-01-01') & (energy_data_df['date'] < '2015-01-01')]
year_2015 = energy_data_df[(energy_data_df['date'] >= '2015-01-01') & (energy_data_df['date'] < '2016-01-01')]
year_2016 = energy_data_df[(energy_data_df['date'] >= '2016-01-01') & (energy_data_df['date'] < '2017-01-01')]
year_2017 = energy_data_df[(energy_data_df['date'] >= '2017-01-01') & (energy_data_df['date'] < '2018-01-01')]
year_2018 = energy_data_df[(energy_data_df['date'] >= '2018-01-01') & (energy_data_df['date'] < '2019-01-01')]
year_2019 = energy_data_df[(energy_data_df['date'] >= '2019-01-01') & (energy_data_df['date'] < '2020-01-01')]
year_2020 = energy_data_df[(energy_data_df['date'] >= '2020-01-01') & (energy_data_df['date'] < '2021-01-01')]


# Select only numerical columns (excluding the 'date' column)
numerical_columns = energy_data_df.select_dtypes(include=['int64', 'float64'])

# Calculate the sum of each numerical column
sum_of_columns_2010 = year_2010[numerical_columns.columns].sum()
sum_of_columns_2011 = year_2011[numerical_columns.columns].sum()
sum_of_columns_2012 = year_2012[numerical_columns.columns].sum()
sum_of_columns_2013 = year_2013[numerical_columns.columns].sum()
sum_of_columns_2014 = year_2014[numerical_columns.columns].sum()
sum_of_columns_2015 = year_2015[numerical_columns.columns].sum()
sum_of_columns_2016 = year_2016[numerical_columns.columns].sum()
sum_of_columns_2017 = year_2017[numerical_columns.columns].sum()
sum_of_columns_2018 = year_2018[numerical_columns.columns].sum()
sum_of_columns_2019 = year_2019[numerical_columns.columns].sum()
sum_of_columns_2020 = year_2020[numerical_columns.columns].sum()


# Create a dictionary with the sum of columns for each year
data = {
    '2010_Totals': sum_of_columns_2010,
    '2011_Totals': sum_of_columns_2011,
    '2012_Totals': sum_of_columns_2012,
    '2013_Totals': sum_of_columns_2013,
    '2014_Totals': sum_of_columns_2014,
    '2015_Totals': sum_of_columns_2015,
    '2016_Totals': sum_of_columns_2016,
    '2017_Totals': sum_of_columns_2017,
    '2018_Totals': sum_of_columns_2018,
    '2019_Totals': sum_of_columns_2019,
    '2020_Totals': sum_of_columns_2020,
}

# Create a DataFrame from the dictionary
energy_data_totals_df = pd.DataFrame(data)

# Display the DataFrame
energy_data_totals_df.tail()

,2010_Totals,2011_Totals,2012_Totals,2013_Totals,2014_Totals,2015_Totals,2016_Totals,2017_Totals,2018_Totals,2019_Totals,2020_Totals
description,,,,,,,,,,,
United States : other,12854,14155,13787,13589,13392,13955,13688,13008,12972,13332,12856
United States : all solar,0,0,0,0,28925,39034,54866,77278,93363,106895,130720
United States : small-scale solar photovoltaic,0,0,0,0,11233,14141,18813,23990,29538,34958,41524
United States : all utility-scale solar,1211,1818,4326,9037,17691,24894,36054,53287,63826,71937,89198
United States : all utility-scale solar,1211,1818,4326,9037,17691,24894,36054,53287,63826,71937,89198


In [14]:
energy_data_totals_df.iloc[0]

2010_Totals    4125059
2011_Totals    4100141
2012_Totals    4047765
2013_Totals    4065965
2014_Totals    4093563
2015_Totals    4078714
2016_Totals    4077573
2017_Totals    4035443
2018_Totals    4180987
2019_Totals    4130575
2020_Totals    4009766
Name: United States : all fuels (utility-scale), dtype: int64

In [15]:
renewable_energy_consumption.head()

,MSN,YYYYMM,Value,Column_Order,Description,Unit
0,WDPRBUS,194913,1549.262,1,Wood Energy Production,Trillion Btu
1,WDPRBUS,195013,1562.307,1,Wood Energy Production,Trillion Btu
2,WDPRBUS,195113,1534.669,1,Wood Energy Production,Trillion Btu
3,WDPRBUS,195213,1474.369,1,Wood Energy Production,Trillion Btu
4,WDPRBUS,195313,1418.601,1,Wood Energy Production,Trillion Btu


In [16]:
# Convert 'YYYYMM' column to string
renewable_energy_consumption['YYYYMM'] = renewable_energy_consumption['YYYYMM'].astype(str)

# Filter out rows where 'YYYYMM' ends with '13'
renewable_energy_consumption_df = renewable_energy_consumption[~renewable_energy_consumption['YYYYMM'].str.endswith('13')]

# Change 'YYYYMM' column name to 'Date'
renewable_energy_consumption_df.rename(columns={'YYYYMM':'Date'}, inplace=True)

# Convert the 'Date' column to a datetime format
renewable_energy_consumption_df['Date'] = pd.to_datetime(renewable_energy_consumption_df['Date'].astype(str), format='%Y%m' )
# df['date_column'] = pd.to_datetime(df['date_column'].astype(str), format='%Y%m%d')

# Convert the 'Value' column to a numeric format
renewable_energy_consumption_df['Value'] = renewable_energy_consumption_df['Value'].replace('Not Available', pd.NA)
renewable_energy_consumption_df['Value'] = pd.to_numeric(renewable_energy_consumption_df['Value'], errors='coerce')

renewable_energy_consumption_df.head()

/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_12152/4111504033.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_energy_consumption_df.rename(columns={'YYYYMM':'Date'}, inplace=True)
/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_12152/4111504033.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  renewable_energy_consumption_df['Date'] = pd.to_datetime(renewable_energy_consumption_df['Date'].astype(str), format='%Y%m' )
/var/folders/qh/b72b_cj57xq1mf0p07xjw8j40000gn/T/ipykernel_12152/4111504033.py:15: SettingWithCopyWarning:

,MSN,Date,Value,Column_Order,Description,Unit
24,WDPRBUS,1973-01-01,129.630,1,Wood Energy Production,Trillion Btu
25,WDPRBUS,1973-02-01,117.194,1,Wood Energy Production,Trillion Btu
26,WDPRBUS,1973-03-01,129.763,1,Wood Energy Production,Trillion Btu
27,WDPRBUS,1973-04-01,125.462,1,Wood Energy Production,Trillion Btu
28,WDPRBUS,1973-05-01,129.624,1,Wood Energy Production,Trillion Btu


In [17]:
renewable_energy_consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7956 entries, 24 to 8929
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   MSN           7956 non-null   object        
 1   Date          7956 non-null   datetime64[ns]
 2   Value         7512 non-null   float64       
 3   Column_Order  7956 non-null   int64         
 4   Description   7956 non-null   object        
 5   Unit          7956 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 435.1+ KB


In [18]:
# Filter data for all dates from year >= 2018
filtered_renewables_df = renewable_energy_consumption_df[(renewable_energy_consumption_df['Date'].dt.year >= 2010) & 
                                              (renewable_energy_consumption_df['Date'].dt.year <= 2020)]
# Reset index
clean_filtered_renewables_df = filtered_renewables_df.reset_index(drop=True)

clean_filtered_renewables_df.head()

,MSN,Date,Value,Column_Order,Description,Unit
0,WDPRBUS,2010-01-01,187.178,1,Wood Energy Production,Trillion Btu
1,WDPRBUS,2010-02-01,171.797,1,Wood Energy Production,Trillion Btu
2,WDPRBUS,2010-03-01,187.220,1,Wood Energy Production,Trillion Btu
3,WDPRBUS,2010-04-01,179.128,1,Wood Energy Production,Trillion Btu
4,WDPRBUS,2010-05-01,181.788,1,Wood Energy Production,Trillion Btu


In [19]:
# Extract year from 'Date' column
clean_filtered_renewables_df['Year'] = clean_filtered_renewables_df['Date'].dt.year

# Group by 'Description' and 'Year', and sum the 'Value' column
grouped_renewables_df = clean_filtered_renewables_df.groupby(['MSN', 'Year','Description', 'Unit'])['Value'].sum().reset_index()

# Sort the DataFrame by 'Year' and 'Description'
grouped_renewables_df = grouped_renewables_df.sort_values(by=['Year', 'Description'])
grouped_renewables_df.reset_index(drop=True, inplace=True)
grouped_renewables_df.head(15)

,MSN,Year,Description,Unit,Value
0,BFTCBUS,2010,Biofuels Consumption,Trillion Btu,1820.984
1,BFPRBUS,2010,Biofuels Production,Trillion Btu,1867.880
2,GETCBUS,2010,Geothermal Energy Consumption,Trillion Btu,111.427
3,HVTCBUS,2010,Hydroelectric Power Consumption,Trillion Btu,887.812
4,SOTCBUS,2010,Solar Energy Consumption,Trillion Btu,67.853
5,BMTCBUS,2010,Total Biomass Energy Consumption,Trillion Btu,4505.636
6,BMPRBUS,2010,Total Biomass Energy Production,Trillion Btu,4552.531
7,RETCBUS,2010,Total Renewable Energy Consumption,Trillion Btu,5895.684
8,REPRBUS,2010,Total Renewable Energy Production,Trillion Btu,5942.579
9,WSTCBUS,2010,Waste Energy Consumption,Trillion Btu,467.911


In [23]:
# Export the 'energy_data_totals_df' and the 'grouped_renewables_df' as CSV files
energy_data_totals_df.to_csv('Net_energy_generation_totals.csv', sep=',', index=True, encoding='utf-8')
grouped_renewables_df.to_csv('renewables_energy_annual_consumption.csv', sep=',', index=False, encoding='utf-8')